In [42]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import time

In [43]:
# Función para verificar si una página existe
def pagina_existe(url):
    response = requests.get(url)
    if response.status_code != 200:
        return False
    soup = BeautifulSoup(response.text, 'html.parser')

    no_results = soup.find('div', class_='no-results')
    if no_results:
        return False
    return True

In [44]:
# Función para obtener detalles de un inmueble
def obtener_detalles_inmueble(url_inmueble):
    try:
        response = requests.get(url_inmueble)
        soup = BeautifulSoup(response.text, "html.parser")
        
        try:
            nombre = soup.find("h1", class_="ad-title")
            nombre = nombre.text.strip() if nombre else "Error al acceder al inmueble"
        except:
            nombre = "Error al acceder al inmueble"

        try:
            agencia = soup.find("p", class_="owner-info__name")
            agencia = agencia.find("a").text.strip() if agencia and agencia.find("a") else "Agencia no disponible"
        except:
            agencia = "Agencia no disponible"

        try:
            features_summary = soup.find("ul", class_="features-summary")
            precio_m2 = None
            if features_summary:
                items = features_summary.find_all("li", class_="features-summary__item")
                for item in items:
                    if "€/m²" in item.text:
                        precio_m2 = item.text.strip()
                        break
            precio_m2 = precio_m2 if precio_m2 else "Error al procesar precio m²"
        except:
            precio_m2 = "Error al procesar precio m²"
        
        try:
            precio = soup.find("div", class_="price__value jsPriceValue")
            precio = precio.text.strip() if precio else "Error al procesar precio"
        except:
            precio = "Error al procesar precio"
        
        try:
            superficie = soup.find("span", class_="features__value")
            superficie = superficie.text.strip() if superficie else "Error al procesar superficie"
        except:
            superficie = "Error al procesar superficie"
        
        try:
            actualizacion = soup.find("div", class_="details__block last-update")
            actualizacion = actualizacion.text.strip().replace("Última actualización\n", "").strip() if actualizacion else "Error al procesar actualizacion"
        except:
            actualizacion = "Error al procesar actualizacion"

        try:
            consumo = soup.find_all("span", class_="energy-certificate__tag")
            consumo_etiquetas = []
            for etiqueta in consumo:
                if 'energy-certificate__tag--' in etiqueta['class'][1]:
                    letra_consumo = etiqueta['class'][1].split('--')[1].strip()
                    if letra_consumo in ["a", "b", "c", "d", "e", "f", "g"]: 
                        consumo_etiquetas.append(letra_consumo.upper())
            consumo = ', '.join(consumo_etiquetas) if consumo_etiquetas else "Error al procesar consumo"
        except:
            consumo = "Error al procesar consumo"

        try:
            emisiones = soup.find_all("span", class_="energy-certificate__tag")
            emisiones_etiquetas = []
            for etiqueta in emisiones:
                if 'energy-certificate__tag--' in etiqueta['class'][1]:
                    letra_emisiones = etiqueta['class'][1].split('--')[1].strip()
                    if letra_emisiones in ["a", "b", "c", "d", "e", "f", "g"]:
                        emisiones_etiquetas.append(letra_emisiones.upper())
            emisiones = ', '.join(emisiones_etiquetas) if emisiones_etiquetas else "Error al procesar emisiones"
        except:
            emisiones = "Error al procesar emisiones"

        try:
            caracteristicas = soup.find("div", class_="features-container")
            detalles = {}
            if caracteristicas:
                secciones = caracteristicas.find_all("div", class_="features__content")
                for seccion in secciones:
                    features = seccion.find_all("div", class_="features__feature")
                    for feature in features:
                        label = feature.find("span", class_="features__label").text.strip().replace(":", "")
                        value = feature.find("span", class_="features__value").text.strip() if feature.find("span", "features__value") else "Si"
                        detalles[label] = value
        except:
            detalles = {}

        # Aquí vamos a extraer el Código Postal e Identificador mediante la URL de cada inmueble
        try:
            codigo_postal_match = re.search(r'(\d{5})-', url_inmueble)
            codigo_postal = codigo_postal_match.group(1) if codigo_postal_match else 'NaN'
        except Exception as e:
            codigo_postal = f"Error al extraer código postal: {e}"
        
        try:
            identificador_match = re.search(r'-(\d+_\d+)', url_inmueble)
            identificador = identificador_match.group(1) if identificador_match else 'NaN'
        except Exception as e:
            identificador = f"Error al extraer identificador: {e}"
        
        # Aquí extraemos el timestamp de cada extracción de datos, que nos será útil a la hora de actualizar la db
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Aquí extraemos las coordenadas (latitud y longitud)
        try:
            location_div = soup.find("div", class_="location")
            data_params = location_div["data-params"]
            
            lat_match = re.search(r'latitude=([-0-9.]+)', data_params)
            lon_match = re.search(r'longitude=([-0-9.]+)', data_params)
            
            latitud = lat_match.group(1) if lat_match else 'NaN'
            longitud = lon_match.group(1) if lon_match else 'NaN'
            
            coordenadas = f"{latitud}, {longitud}"
        except Exception as e:
            coordenadas = f"Error al extraer coordenadas: {e}"
            
        return nombre, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, detalles, codigo_postal, identificador, timestamp, coordenadas
    except Exception as e:
        return ("Error al acceder al inmueble",) * 9 + ({}, "", "", datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "")

In [45]:
# Función para obtener inmuebles de URLs
def obtener_inmuebles_de_urls(urls_a_scrapear):
    todos_los_inmuebles = []
    columnas = set()

    for url_pagina in urls_a_scrapear:
        print(f"Procesando la URL: {url_pagina}")
        
        pagina = 1
        while True:
            try:
                response = requests.get(url_pagina)
                if response.status_code == 404:
                    print(f"Página no encontrada: {url_pagina}")
                    break
                soup = BeautifulSoup(response.text, "html.parser")
                
                titulos_soup = soup.find_all("a", class_="ad-preview__title")
                
                for titulo_soup in titulos_soup:
                    nombre = titulo_soup.text.strip()
                    href = "https://www.pisos.com" + titulo_soup['href']
                    nombre_detalle, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, detalles, codigo_postal, identificador, timestamp, coordenadas = obtener_detalles_inmueble(href)
                    
                    # corregido extraer ubic
                    ubicacion = url_pagina.split('/')[-3].split('-')[-1]
                    
                    inmueble = {
                        "nombre": nombre,
                        "agencia": agencia,
                        "precio_m2": precio_m2,
                        "precio": precio,
                        "superficie": superficie,
                        "href": href,
                        "actualizacion": actualizacion,
                        "consumo": consumo,
                        "emisiones": emisiones,
                        "ubicacion": ubicacion,
                        "codigo_postal": codigo_postal,
                        "identificador": identificador,
                        "timestamp": timestamp,
                        "coordenadas": coordenadas
                    }
                    
                    inmueble.update(detalles)
                    todos_los_inmuebles.append(inmueble)
                    columnas.update(inmueble.keys())
                
                pagina += 1
                if pagina > 2:  # Limitar a 2 páginas para la prueba
                    break
                url_pagina = f"{url_pagina.rsplit('/', 2)[0]}/{pagina}/"
            
            except Exception as e:
                print(f"Error al procesar la URL {url_pagina}: {e}")
                break
            time.sleep(1) 
    
    df_inmuebles = pd.DataFrame(todos_los_inmuebles, columns=list(columnas))
    return df_inmuebles


In [46]:
# Leer el archivo txt con las URLs y asignarlas a la lista 'urls'
file_path = 'urls_a_scrapear.txt'
try:
    with open(file_path, 'r') as file:
        urls = file.read().splitlines()
    print(f"Se han cargado {len(urls)} URLs desde el archivo.")
except FileNotFoundError:
    print(f"El archivo {file_path} no se encontró.")

Se han cargado 437 URLs desde el archivo.


In [48]:
# Generar lista de URLs a scrapear
urls_a_scrapear = []

for base_url in urls:     # Intentar si hay hasta la página 100
    for i in range(1, 101):
        url = f"{base_url}{i}/"
        if pagina_existe(url):
            urls_a_scrapear.append(url)
        else:
            print(f"Página no encontrada: {url}")
            break  # Si no existe la página, lo dejo 
print(f"Hay {len(urls_a_scrapear)} URLs a scrapear")

Página no encontrada: https://www.pisos.com/venta/pisos-corredor_del_henares/20/
Página no encontrada: https://www.pisos.com/venta/pisos-madrid_noroeste/45/
Página no encontrada: https://www.pisos.com/venta/pisos-madrid_norte/31/
Página no encontrada: https://www.pisos.com/venta/pisos-madrid_sur/37/
Página no encontrada: https://www.pisos.com/venta/pisos-madrid_sureste/10/
Página no encontrada: https://www.pisos.com/venta/pisos-madrid_suroeste/19/
Página no encontrada: https://www.pisos.com/venta/pisos-arganzuela/5/
Página no encontrada: https://www.pisos.com/venta/pisos-pisos-madrid_capital_barajas/3/
Página no encontrada: https://www.pisos.com/venta/pisos-barrio_de_salamanca/27/
Página no encontrada: https://www.pisos.com/venta/pisos-madrid_capital_carabanchel/7/
Página no encontrada: https://www.pisos.com/venta/pisos-madrid_capital_centro/29/
Página no encontrada: https://www.pisos.com/venta/pisos-madrid_capital_chamartin/10/
Página no encontrada: https://www.pisos.com/venta/pisos-c

In [49]:
# Obtener inmuebles de las URLs generadas
df_inmuebles = obtener_inmuebles_de_urls(urls_a_scrapear)

Procesando la URL: https://www.pisos.com/venta/pisos-corredor_del_henares/1/
Procesando la URL: https://www.pisos.com/venta/pisos-corredor_del_henares/2/
Procesando la URL: https://www.pisos.com/venta/pisos-corredor_del_henares/3/
Procesando la URL: https://www.pisos.com/venta/pisos-corredor_del_henares/4/
Procesando la URL: https://www.pisos.com/venta/pisos-corredor_del_henares/5/
Procesando la URL: https://www.pisos.com/venta/pisos-corredor_del_henares/6/
Procesando la URL: https://www.pisos.com/venta/pisos-corredor_del_henares/7/
Procesando la URL: https://www.pisos.com/venta/pisos-corredor_del_henares/8/
Procesando la URL: https://www.pisos.com/venta/pisos-corredor_del_henares/9/
Procesando la URL: https://www.pisos.com/venta/pisos-corredor_del_henares/10/
Procesando la URL: https://www.pisos.com/venta/pisos-corredor_del_henares/11/
Procesando la URL: https://www.pisos.com/venta/pisos-corredor_del_henares/12/
Procesando la URL: https://www.pisos.com/venta/pisos-corredor_del_henares

In [50]:
df_inmuebles.columns

Index(['Antigüedad', 'identificador', 'Teléfono', 'Carpintería interior',
       'Ascensor', 'Portero automático', 'Armarios empotrados', 'Conservación',
       'Superficie construida', 'Lavadero', 'Calle alumbrada', 'Calefacción',
       'Calle asfaltada', 'codigo_postal', 'Referencia',
       'Sistema de seguridad', 'Aire acondicionado', 'Garaje',
       'Cocina equipada', 'agencia', 'Terraza', 'Puerta blindada',
       'Tipo de casa', 'Gas', 'href', 'Gastos de comunidad',
       'Se aceptan mascotas', 'Adaptado a personas con movilidad reducida',
       'Urbanizado', 'Comedor', 'Soleado', 'Habitaciones', 'nombre',
       'ubicacion', 'Exterior', 'Jardín', 'Superficie solar', 'Planta',
       'Balcón', 'superficie', 'actualizacion', 'emisiones', 'Vidrios dobles',
       'Orientación', 'precio', 'Luz', 'consumo', 'Trastero', 'coordenadas',
       'Superficie útil', 'Agua', 'Baños', 'Tipo suelo', 'Chimenea',
       'Alcantarillado', 'precio_m2', 'timestamp', 'Carpintería exterior',
   

In [51]:
df_seleccionado = df_inmuebles[["codigo_postal", "nombre", "ubicacion", "actualizacion", "precio", "timestamp", "coordenadas"]]

In [52]:
df_seleccionado

,codigo_postal,nombre,ubicacion,actualizacion,precio,timestamp,coordenadas
0,NaN,"Casa adosada en calle de la Gran Vía, 7",corredor_del_henares,Anuncio actualizado el 28/02/2025,323.300 €,2025-02-28 21:02:41,"40.5438, -3.37927"
1,NaN,"Casa adosada en calle de la Gran Vía, 12",corredor_del_henares,Anuncio actualizado el 08/02/2025,340.200 €,2025-02-28 21:02:42,"40.54455876322138, -3.3799266815185547"
2,NaN,Casa adosada en Villalbilla,corredor_del_henares,Anuncio actualizado el 29/01/2025,322.000 €,2025-02-28 21:02:42,"40.4464284, -3.3478297"
3,NaN,"Casa adosada en calle de la Gran Vía, 12",corredor_del_henares,Anuncio actualizado el 08/02/2025,338.370 €,2025-02-28 21:02:42,"40.54455876322138, -3.3799266815185547"
4,NaN,Chalet unifamiliar en calle de Soto del Henares,corredor_del_henares,Anuncio actualizado el 13/02/2025,320.000 €,2025-02-28 21:02:42,"40.4304711, -3.2924094"
...,...,...,...,...,...,...,...
261929,49004,Piso en Centro,zamora_tierra_del_pan,Anuncio actualizado el 02/02/2025,230.000 €,2025-03-01 17:07:38,"41.5050128, -5.7468752"
261930,49028,Piso en Pinilla,zamora_tierra_del_pan,Anuncio actualizado el 12/02/2025,128.000 €,2025-03-01 17:07:39,"41.496715487, -5.740188438"
261931,49025,Casa en San Lazaro,zamora_tierra_del_pan,Anuncio actualizado el 29/01/2025,50.000 €,2025-03-01 17:07:39,"41.5091328, -5.75328"
261932,49021,Piso en Los Bloques,zamora_tierra_del_pan,Anuncio actualizado el 13/02/2025,145.000 €,2025-03-01 17:07:39,"41.5117874, -5.7309068"


In [ ]:
df_inmuebles.to_csv('compras_completo.csv', index=False)